# Preparación del modelo

## Importación de librerías

In [17]:
import glob, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio as rio
import seaborn as sns

# Carga de datos

## Geodataframe

In [18]:
agebs = gpd.read_file('data/colima/agebs.gpkg')
agebs["area"] = agebs.area / 1000 # ha
agebs.drop(columns=["CVE_ENT","CVE_MUN", "CVE_AGEB","CVE_LOC","geometry"], inplace=True)
agebs = pd.DataFrame(agebs)

## Zonal Statistics

In [26]:
df0 = pd.read_csv('data/megaGDF.csv')
df0.drop(columns=["CVE_ENT","CVE_MUN","CVE_LOC", "_count","_sum"], inplace=True)
df0.head()

,CVEGEO,CVE_AGEB,_mean,class,time
0,0600200010676,0676,NaN,Depth,0
1,0600200010727,0727,NaN,Depth,0
2,0600200011142,1142,NaN,Depth,0
3,0600200010411,0411,NaN,Depth,0
4,0600200011068,1068,NaN,Depth,0


In [41]:
df = df0.pivot_table(index=["CVEGEO",'CVE_AGEB',"time"], columns='class', values=["_mean"])
df.reset_index(inplace=True)
# remove double header
df.columns = df.columns.droplevel(0)
df.columns = ['CVEGEO', 'CVE_AGEB',"time", 'Depth', 'Rain', 'Velocity']
df

,CVEGEO,CVE_AGEB,time,Depth,Rain,Velocity
0,0600200010318,0318,0,NaN,0.00,NaN
1,0600200010318,0318,300,NaN,0.00,NaN
2,0600200010318,0318,600,NaN,0.72,NaN
3,0600200010318,0318,900,NaN,0.72,NaN
4,0600200010318,0318,1200,NaN,1.38,NaN
...,...,...,...,...,...,...
2123,0601000010362,0362,23100,0.016889,4.80,0.070683
2124,0601000010362,0362,23400,0.016851,8.76,0.070517
2125,0601000010362,0362,23700,0.016796,8.76,0.070501
2126,0601000010362,0362,24000,0.016748,7.80,0.070419


In [42]:
df = df.merge(agebs, on="CVEGEO")
df

,CVEGEO,CVE_AGEB,time,Depth,Rain,Velocity,area
0,0600200010318,0318,0,NaN,0.00,NaN,339.556144
1,0600200010318,0318,300,NaN,0.00,NaN,339.556144
2,0600200010318,0318,600,NaN,0.72,NaN,339.556144
3,0600200010318,0318,900,NaN,0.72,NaN,339.556144
4,0600200010318,0318,1200,NaN,1.38,NaN,339.556144
...,...,...,...,...,...,...,...
2123,0601000010362,0362,23100,0.016889,4.80,0.070683,227.156848
2124,0601000010362,0362,23400,0.016851,8.76,0.070517,227.156848
2125,0601000010362,0362,23700,0.016796,8.76,0.070501,227.156848
2126,0601000010362,0362,24000,0.016748,7.80,0.070419,227.156848


## Denue and INV

In [43]:
denue_inv = pd.read_csv('data/denue_inv.csv')
denue_inv.head()

,CVE_AGEB,POBTOT,POB0_14_P,P15A29A_P,P30A59A_P,GRAPROES,P_CD_P,VPH_C_EL_P,VPH_EXSA_P,VPH_DREN_P,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
0,0273,74,6.349231,7.081538,13.980769,3.403077,3.540769,30.769231,29.486923,400.00,...,39,39,38,31,39,39,39,1.0,0.0,0.0
1,0318,1925,11.397551,17.568980,31.385102,10.333878,7.330000,85.303878,85.473878,4188.22,...,143,128,115,81,144,146,143,13.0,6.0,0.0
2,0322,2143,11.647736,17.747170,30.118113,11.442453,7.085472,88.486038,88.553396,4693.33,...,155,150,144,70,159,155,151,13.0,3.0,3.0
3,0362,2342,18.422286,21.292000,33.820000,9.617714,5.100857,91.145714,91.380857,3198.33,...,103,102,97,45,105,102,102,4.0,1.0,1.0
4,0411,1116,9.866053,17.035000,33.167105,11.912895,4.348947,81.578947,81.578947,3100.00,...,113,112,109,66,47,113,111,12.0,2.0,0.0


## Join

In [44]:
df_full = df.merge(denue_inv, on="CVE_AGEB")
df_full

,CVEGEO,CVE_AGEB,time,Depth,Rain,Velocity,area,POBTOT,POB0_14_P,P15A29A_P,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
0,0600200010318,0318,0,NaN,0.00,NaN,339.556144,1925,11.397551,17.56898,...,143,128,115,81,144,146,143,13.0,6.0,0.0
1,0600200010318,0318,300,NaN,0.00,NaN,339.556144,1925,11.397551,17.56898,...,143,128,115,81,144,146,143,13.0,6.0,0.0
2,0600200010318,0318,600,NaN,0.72,NaN,339.556144,1925,11.397551,17.56898,...,143,128,115,81,144,146,143,13.0,6.0,0.0
3,0600200010318,0318,900,NaN,0.72,NaN,339.556144,1925,11.397551,17.56898,...,143,128,115,81,144,146,143,13.0,6.0,0.0
4,0600200010318,0318,1200,NaN,1.38,NaN,339.556144,1925,11.397551,17.56898,...,143,128,115,81,144,146,143,13.0,6.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,0601000010362,0362,23100,0.016889,4.80,0.070683,227.156848,2342,18.422286,21.29200,...,103,102,97,45,105,102,102,4.0,1.0,1.0
2124,0601000010362,0362,23400,0.016851,8.76,0.070517,227.156848,2342,18.422286,21.29200,...,103,102,97,45,105,102,102,4.0,1.0,1.0
2125,0601000010362,0362,23700,0.016796,8.76,0.070501,227.156848,2342,18.422286,21.29200,...,103,102,97,45,105,102,102,4.0,1.0,1.0
2126,0601000010362,0362,24000,0.016748,7.80,0.070419,227.156848,2342,18.422286,21.29200,...,103,102,97,45,105,102,102,4.0,1.0,1.0


In [45]:
df_full.describe()

,time,Depth,Rain,Velocity,area,POBTOT,POB0_14_P,P15A29A_P,P30A59A_P,GRAPROES,...,PARATRAN_C,DRENAJEP_C,TRANSCOL_C,ARBOLES_C,ACESOAUT_C,PUESSEMI_C,PUESAMBU_C,escuela,farmacia,hospital
count,2128.000000,1509.000000,2106.000000,1465.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,...,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000,2128.000000
mean,12127.161654,0.264942,5.194074,0.766056,407.738402,1829.714756,13.715332,18.723482,33.262518,10.481927,...,111.604793,106.189850,101.108553,59.328477,105.552162,112.220865,110.532425,8.067199,2.764568,0.921053
std,7089.632672,0.233180,7.023082,0.627469,119.083711,760.404175,4.298687,2.715630,3.538077,1.518658,...,24.507491,26.358236,23.282044,15.599136,29.154342,24.388624,23.871086,5.945742,2.782429,1.171423
min,0.000000,0.010123,0.000000,0.006398,205.589137,311.000000,5.978372,13.757209,26.724800,7.758302,...,72.000000,59.000000,60.000000,33.000000,47.000000,75.000000,75.000000,2.000000,0.000000,0.000000
25%,6000.000000,0.084680,1.140000,0.250919,339.556144,1410.000000,10.981154,17.035000,30.118113,9.569600,...,92.000000,84.000000,87.000000,45.000000,81.000000,93.000000,91.000000,3.000000,1.000000,0.000000
50%,12000.000000,0.184516,2.220000,0.626592,417.701078,1935.000000,12.719286,19.053636,33.346176,10.210981,...,112.000000,108.000000,101.000000,58.500000,102.000000,113.000000,111.000000,6.000000,2.000000,1.000000
75%,18300.000000,0.393369,7.200000,1.045419,457.692900,2143.000000,15.677234,20.527500,35.534118,11.442453,...,129.000000,127.000000,119.000000,67.000000,128.000000,129.000000,129.000000,12.000000,3.000000,1.000000
max,24300.000000,1.005461,36.300000,2.407312,715.165608,4375.000000,25.316415,25.398293,40.178511,13.536364,...,159.000000,159.000000,151.000000,94.000000,159.000000,159.000000,158.000000,26.000000,14.000000,4.000000
